In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 200 kB 49.4 MB/s 
     |████████████████████████████████| 199 kB 58.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=5f3b77df0f8d2344609de55064ce7f7d73cd3c8eedc8e6e56bd4462af25a4733
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [7]:
df = spark.read.csv("/content/drive/MyDrive/SparkWork/SparkDoc/ContainsNull.txt", inferSchema=True, header=True)

In [8]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# ***Three options in handling a dataset with missing values***

* Keep Missing datapoints as null
* Droping Missing data with the entire row
* Filling data with some value

".na" methode can be used to fill/ drop / do something of that missing data.

In [9]:
df.na.drop().show() # drops every row that has a missing value

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.drop(thresh=2).show()#drops every row that has atleast 2 null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+





*   Theres also another argument that can be passed into the na.drop() and that is how, and its by default set to "any" thats why using drop alone deletes entire rows that have atleast a null value.
*   thers another option which is 'all' -> will remove the entire row if only the entire row only consists null values.



In [11]:
df.na.drop(how = "all").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



***subsets-> specifies which column are we conidering as a base for removing unwanted data.***

In [13]:
df.na.drop(subset=['sales']).show() # removes the entire row if sales column is missing any value

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## ***Filling Data***
* df.na.fill()
* spark can fill in and match up data types.
eg: in case if we do df.na.fill('Fill Value').show() => will fill all the missing values that are a string type column
* df.na.fill(0).show() will fill 0 insted of null in integer type etc.
* We can also specify the column name to wich we are filling the data.
eg: df.na.fill('No Name', subset=['name']).show() => results in filling "No Name" in only name column.


In [14]:
df.na.fill('Fill Value').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Fill Value| null|
|emp3|Fill Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [17]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [18]:
df.na.fill('No Name', subset= ['name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



## ***Filling mean value***

In [19]:
from pyspark.sql.functions import mean

In [20]:
mean_val = df.select(mean(df['sales'])).collect()

In [24]:
mean_sales = mean_val[0][0]

In [25]:
df.na.fill(mean_sales, ['sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

